<a href="https://colab.research.google.com/github/PhilosophicalBlackhole/Excel-ImageURL-Insert/blob/main/Python_Script_LLM_Video_Analysis_for_Dynamic_Cropping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

--2025-06-20 22:31:46--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2025-06-20 22:31:46 (59.6 MB/s) - ‘yolov3.cfg’ saved [8342/8342]



In [2]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2025-06-20 22:31:57--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 172.67.185.199, 104.21.88.156, 2606:4700:3037::6815:589c, ...
Connecting to pjreddie.com (pjreddie.com)|172.67.185.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘yolov3.weights’

yolov3.weights          [ <=>                ]   8.88K  --.-KB/s    in 0s      

2025-06-20 22:31:58 (57.7 MB/s) - ‘yolov3.weights’ saved [9093]



In [3]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

--2025-06-20 22:32:04--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2025-06-20 22:32:05 (28.4 MB/s) - ‘coco.names’ saved [625/625]



In [5]:
import cv2
import numpy as np
!pip install opencv-python
# You will need to download yolov3.cfg, yolov3.weights, and coco.names separately
# !wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
# !wget https://pjreddie.com/media/files/yolov3.weights
# !wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

# --- Model Configuration ---
# Replace with the actual paths to your model configuration and weights files
# You can download YOLO files from various sources, e.g., https://github.com/pjreddie/darknet
config_path = "yolov3.cfg"
weights_path = "yolov3.weights"
labels_path = "coco.names" # Optional: file with class names

# Load the DNN model
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# Get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class names (if available)
classes = None
with open(labels_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# --- Video Processing (Modified from your original code) ---
cap = cv2.VideoCapture("input_video.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer for cropped output (adjust dimensions if needed)
crop_width, crop_height = 400, 400  # example crop size around the subject
out = cv2.VideoWriter("cropped_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (crop_width, crop_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for the model
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process the model output (example: find a dominant object like a person or window)
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5: # Confidence threshold
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression to remove redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) # Confidence and NMS thresholds

    # Find the bounding box for the object you are interested in (e.g., a window)
    # This part requires specific logic based on what you want to detect.
    # For example, if you are looking for a "window" object and it's class ID 10:
    target_bbox = None
    if classes:
        try:
            window_class_id = classes.index("window") # Or whatever your target object is
            for i in indexes:
                if class_ids[i] == window_class_id:
                    target_bbox = boxes[i]
                    break # Found the target object, take the first one

        except ValueError:
            print("Warning: 'window' class not found in coco.names. Adapt this part to your target object.")
            # As a fallback, you might just take the largest bounding box or the one with highest confidence
            if len(indexes) > 0:
                 best_index = indexes[0] # Example: take the first one after NMS
                 target_bbox = boxes[best_index]


    # If a target bounding box is found, crop the frame
    if target_bbox is not None:
        x = max(0, x)
        y = max(0, y)
        w = min(w, width - x)
        h = min(h, height - y)

        cropped_frame = frame[y:y+h, x:x+w]

        # Resize cropped frame to uniform output size
        if cropped_frame.shape[0] > 0 and cropped_frame.shape[1] > 0: # Check if crop is valid
            cropped_frame = cv2.resize(cropped_frame, (crop_width, crop_height))
            # Write to output video
            out.write(cropped_frame)
        else:
             print(f"Skipping frame due to invalid crop dimensions: {cropped_frame.shape}")


cap.release()
out.release()
cv2.destroyAllWindows() # Close any OpenCV windows

error: OpenCV(4.11.0) /io/opencv/modules/dnn/src/darknet/darknet_io.cpp:991: error: (-213:The function/feature is not implemented) Transpose the weights (except for convolutional) is not implemented in function 'ReadDarknetFromWeightsStream'
